# Recommendations
- 유툽 추천알고리즘, 온라인쇼핑몰 등
1. 콘텐츠 기반 필터링 추천 시스템: 사용자가 특정 아이템을 선호하는 경오, 그와 비슷한 아이템을 추천
2. 최근접 이웃 협업 필터링: 축적된 사용자 행동 데이터를 기반으로 사용자가 아직 평가하지 않은 아이템을 예측평가.
　　　　　　　　　　　　　　　　　　　　(사용자기반: 당신과 비슷한 고객들이 다음 상픔도 구매했다.)


　　- 일반적으론 사용자 기반보다 아이템 기반 협업 필터링이 정확도가 더 높다.



3. 잠재요인 협업 필터링: 사용자-아이템 평점 행렬 데이터를 이용해 "잠재요인"을 도출하는 것. 주요인과 아이템에 대한 잠재요인데 애해 행렬분해를 하고, 다시 행렬곱을 통해 아직 평점을 부여하지 않은 아이템에 대한 예측 평점을 생성하는 것

---
# 콘텐츠 기반 필터링 실습 - TMDB5000 영화 데이터 셋
- [캐글 페이지](https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata)

## 1. 데이터 불러오기

In [61]:
import pandas as pd
import numpy as np

In [62]:
movies = pd.read_csv('https://raw.githubusercontent.com/PinkWink/ML_tutorial/master/dataset/tmdb_5000_movies.csv')
print(movies.shape)

(4803, 20)


In [63]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [64]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

## 2. 데이터 선택

In [65]:
movies_df = movies[['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']]
movies_df.head()

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",6.9,4500,139.082615,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.3,4466,107.376788,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",7.6,9106,112.312950,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.1,2124,43.926995,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca..."


## 3. 데이터 정리(1) 
- 'genres' & 'keywords' 컬럼의 dict으로 이뤄진 리스트에서 필요 부분만 남기기 

In [66]:
movies_df['keywords'][0]

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

- 'genres'와 'keywords' 는 컬럼 안에 dict 형으로 저장되어있다.

In [67]:
type(movies_df['keywords'][0])

str

- 타입은 str

#### 　 literal_eval 은 문자열로 된 데이터를 튜플로 변환해준다.

In [68]:
from ast import literal_eval

In [69]:
code = """(1, 2, {'foo': 'bar'})"""
code

"(1, 2, {'foo': 'bar'})"

In [70]:
type(code)

str

In [71]:
literal_eval(code)

(1, 2, {'foo': 'bar'})

In [72]:
type(literal_eval(code))

tuple

#### 　'genres' & 'keywords' 컬럼을 list와 dict으로 복구

In [73]:
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

movies_df.head()

C:\Users\user\AppData\Local\Temp\ipykernel_15196\2327736922.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['genres'] = movies_df['genres'].apply(literal_eval)
C:\Users\user\AppData\Local\Temp\ipykernel_15196\2327736922.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)


,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",7.2,11800,150.437577,"[{'id': 1463, 'name': 'culture clash'}, {'id':...","In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",6.9,4500,139.082615,"[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...","Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",6.3,4466,107.376788,"[{'id': 470, 'name': 'spy'}, {'id': 818, 'name...",A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",7.6,9106,112.312950,"[{'id': 849, 'name': 'dc comics'}, {'id': 853,...",Following the death of District Attorney Harve...
4,49529,John Carter,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",6.1,2124,43.926995,"[{'id': 818, 'name': 'based on novel'}, {'id':...","John Carter is a war-weary, former military ca..."


In [74]:
type(movies_df['keywords'][0])

list

#### 　dict의 value값을 특성으로 사용하도록 변경

In [75]:
movies_df['genres'][0]

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [76]:
[y['name'] for y in movies_df['genres'][0]]

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [77]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : [y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [y['name'] for y in x])

C:\Users\user\AppData\Local\Temp\ipykernel_15196\3997152784.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['genres'] = movies_df['genres'].apply(lambda x : [y['name'] for y in x])
C:\Users\user\AppData\Local\Temp\ipykernel_15196\3997152784.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [y['name'] for y in x])


```
lambda x : [y['name'] for y in x] 
: dict으로 이뤄진 list에서 하나씩 받아 키 'name' 의 value 만 추출해 리스트로 저장
```

In [78]:
movies_df[['keywords' , 'genres']][:2]

,keywords,genres
0,"[culture clash, future, space war, space colon...","[Action, Adventure, Fantasy, Science Fiction]"
1,"[ocean, drug abuse, exotic island, east india ...","[Adventure, Fantasy, Action]"


## 3. 데이터 정리(2)
- 'genres' 컬럼에서 각 단어들을 하나의 문장(띄어쓰기로 구분된)으로 변환

In [79]:
movies_df['genres'][0]

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [80]:
(' ').join(movies_df['genres'][0])

'Action Adventure Fantasy Science Fiction'

In [81]:
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))

movies_df.head()

C:\Users\user\AppData\Local\Temp\ipykernel_15196\4154314993.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))


,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy Science Fiction
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",6.9,4500,139.082615,"[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",Adventure Fantasy Action
2,206647,Spectre,"[Action, Adventure, Crime]",6.3,4466,107.376788,"[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...,Action Adventure Crime
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",7.6,9106,112.312950,"[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...,Action Crime Drama Thriller
4,49529,John Carter,"[Action, Adventure, Science Fiction]",6.1,2124,43.926995,"[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca...",Action Adventure Science Fiction


## 4. 문자열로 변환된 genres를 CountVectorize 수행

### 　코사인 유사도
$$
similarity = \cos(\theta) 
= \frac{A\, \cdot \,B}{\lvert \lvert A \lvert \lvert \,  \lvert \lvert B \vert \vert} 
= \frac{\displaystyle\sum_{i=1}^{n}{A_i \times B_i}}{\sqrt{\displaystyle\sum_{i=1}^{n}{(A_i)^2}} \times \sqrt{\displaystyle\sum_{i=1}^{n}{(B_i)^2}}}
$$






    - 반대 방향 : -1
    - 90도 직각 : 0
    - 같은 방향 : 1

#### 벡터 만들기

In [82]:
from sklearn.feature_extraction.text import CountVectorizer

In [83]:
count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


```
예시 : Action Crime Drama Thriller
- ngram_range = 1 : Action Crime Drama Thriller 각각 다른 것으로 본다.
- ngram_range = 2 : Action Crime Drama Thriller 각각 & 2개씩 묶어서(ex.Action Crime, Action Thriller 같이)도 따로 본다.
```

#### 코사인 유사도 측정

In [84]:
from sklearn.metrics.pairwise import cosine_similarity

In [85]:
genre_sim = cosine_similarity(genre_mat, genre_mat)

print(genre_sim.shape)
print(genre_sim[:2])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]


#### genre_sim 에서 높은 값 순으로 정렬

In [86]:
genre_sim_sorted = genre_sim.argsort()[:, ::-1] # numpy엔 asc 기능이 따로 없음.  ::-1 라고 적으면 된다.
genre_sim_sorted[:1]  # 인덱스값을 출력!

array([[   0, 3494,  813, ..., 3038, 3037, 2401]], dtype=int64)

## 5. 영화 추천 

### 추천 영화를 DF로 반환하는 함수

In [87]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    
    title_index = title_movie.index.values
    similar_indexs = sorted_ind[title_index, :(top_n)]
    
    print(similar_indexs)
    similar_indexs = similar_indexs.reshape(-1)
    
    return df.iloc[similar_indexs]

### "대부" 와 비슷한 영화 알아보기

In [88]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted, 'The Godfather', 10)
similar_movies[['title', 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


- Mi America 의 평점 평균이 0.0 ..?
- 뭔가 이상한 데이터 발견

In [89]:
movies_df[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


- 평점과 평점을 매긴 횟수를 보면 문제 데이터가 보인다.
- ex) 한명이 만점을 준 경우, 신뢰하기 어렵다

## 6. 영화 선정을 위한 가중치 설정
- 위에서 문제 데이터 발견

## $$ {(\frac{v}{v+m})R} + {(\frac{m}{v+m})C} $$




```
　　　　　- v : 개별 영화에 평점을 투표한 횟수
　　　　　- m : 평점을 부여하기 위한 최소 투표 횟수
　　　　　- R : 개별 영화에 대한 평균 평점
　　　　　- C : 전체 영화에 대한 평균 평점
```

### 영화 전체 평균평점과 최소 투표 횟수를 60% 지점으로 설정

In [94]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)

print('C:', round(C, 3), ', m: ',round(m, 3))

C: 6.092 , m:  370.2


### 가중치가 부여된 평점을 계산하기 위한 함수

In [96]:
def weighted_vote_aversge(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ((v / (v + m)) * R) + ((m / (m+v)) * C)

In [97]:
movies_df['weighted_vote'] = movies_df.apply(weighted_vote_aversge, axis=1)
movies_df.head()

C:\Users\user\AppData\Local\Temp\ipykernel_15196\826825452.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['weighted_vote'] = movies_df.apply(weighted_vote_aversge, axis=1)


,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal,weighted_vote
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy Science Fiction,7.166301
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",6.9,4500,139.082615,"[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",Adventure Fantasy Action,6.838594
2,206647,Spectre,"[Action, Adventure, Crime]",6.3,4466,107.376788,"[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...,Action Adventure Crime,6.284091
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",7.6,9106,112.312950,"[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...,Action Crime Drama Thriller,7.541095
4,49529,John Carter,"[Action, Adventure, Science Fiction]",6.1,2124,43.926995,"[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca...",Action Adventure Science Fiction,6.098838


In [98]:
movies_df[movies_df['vote_count'] < 10]

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal,weighted_vote
463,161795,Déjà Vu,"[Romance, Drama]",8.0,1,0.605645,"[love, american, pin, stranger, ruby]",L.A. shop owner Dana and Englishman Sean meet ...,Romance Drama,6.097311
492,293644,Top Cat Begins,"[Comedy, Animation]",5.3,9,0.719996,[3d],Top Cat has arrived to charm his way into your...,Comedy Animation,6.073370
1023,7504,Earth,[Drama],6.6,9,1.246883,"[based on novel, war of independence, period d...",It's 1947 and the borderlines between India an...,Drama,6.104224
1039,113464,Inchon,"[Drama, History, War]",6.5,2,0.146783,[],A noisy and absurd re-telling of the great 195...,Drama History War,6.094363
1453,49478,Warriors of Virtue,"[Fantasy, Family, Action]",4.7,9,0.912395,"[american football, mythology, chinese food, k...","A young man, Ryan, suffering from a disability...",Fantasy Family Action,6.059130
...,...,...,...,...,...,...,...,...,...,...
4795,124606,Bang,[Drama],6.0,1,0.918116,"[gang, audition, police fake, homeless, actress]",A young woman in L.A. is having a bad day: she...,Drama,6.091923
4797,67238,Cavite,"[Foreign, Thriller]",7.5,2,0.022173,[],"Adam, a security guard, travels from Californi...",Foreign Thriller,6.099736
4799,72766,Newlyweds,"[Comedy, Romance]",5.9,5,0.642552,[],A newlywed couple's honeymoon is upended by th...,Comedy Romance,6.089611
4800,231617,"Signed, Sealed, Delivered","[Comedy, Drama, Romance, TV Movie]",7.0,6,1.444476,"[date, love at first sight, narration, investi...","""Signed, Sealed, Delivered"" introduces a dedic...",Comedy Drama Romance TV Movie,6.106650


In [99]:
movies_df[['title', 'vote_average', 'weighted_vote', 'vote_count']].sort_values('weighted_vote', ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


### 유사 영화를 찾는 함수 변경

In [100]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    
    similar_indexs = sorted_ind[title_index, :(top_n * 2)]
    similar_indexs = similar_indexs.reshape(-1)
    
    similar_indexs = similar_indexs[similar_indexs != title_index]
    
    return df.iloc[similar_indexs].sort_values('weighted_vote', ascending=False)[:top_n]

### 다시 "대부" 와 유사한 영화 찾기

In [101]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted, 'The Godfather', 10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427
